#### BOW,TFIDF,Machine Learning Algorithms
(order very imp***)
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW And TF-IDF (Sentences--->vectors) {Preventing Data Leakage}
4. Trained Our Models


In [3]:
import pandas as pd
messages=pd.read_csv('../smsspamcollection/SMSSpamCollection', sep='\t',names=["label","message"])

In [4]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
## Data Cleaning And Preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()

In [74]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
    review=[wnl.lemmatize(word, pos='v') for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [75]:
corpus[0:4]

['go jurong point crazy available bugis n great world la e buffet cine get amore wat',
 'ok lar joke wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say']

In [153]:
## Output Features
y=pd.get_dummies(messages['label'])
print(y.astype(int))
y=y.iloc[:,1].values
y.astype(int)

      ham  spam
0       1     0
1       1     0
2       0     1
3       1     0
4       1     0
...   ...   ...
5567    0     1
5568    1     0
5569    1     0
5570    1     0
5571    1     0

[5572 rows x 2 columns]


array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

In [135]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)
len(X_train),len(y_train)

(4457, 4457)

In [136]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

classifiers = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    "SVM": LinearSVC(class_weight='balanced'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100)
}

In [138]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

scores = []
best_model = None
best_score = 0

for name, clf in classifiers.items():
    print(f"\n🧠 {name}")
    pipe = Pipeline([
        ('cv', CountVectorizer(max_features=3000, stop_words='english', ngram_range=(1, 2))),
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    if acc > best_score:
        best_score = acc
        best_model = (name, pipe)

print("\n🏆 Best Model:")
print(f"{best_model[0]} with accuracy: {best_score:.4f}")


🧠 Naive Bayes
Accuracy: 0.9883408071748879
              precision    recall  f1-score   support

       False       0.99      0.99      0.99       966
        True       0.97      0.95      0.96       149

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115


🧠 Logistic Regression
Accuracy: 0.9811659192825112
              precision    recall  f1-score   support

       False       0.99      0.99      0.99       966
        True       0.94      0.91      0.93       149

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115


🧠 Random Forest
Accuracy: 0.9721973094170404
              precision    recall  f1-score   support

       False       0.97      0.99      0.98       966
        True       0.96      0.83      0.89       149

    accuracy                           0

## Create Bag Of Words

In [139]:
## Create the Bag OF Words model
from sklearn.feature_extraction.text import CountVectorizer
## for Binary BOW enable binary=True
cv=CountVectorizer(max_features=3000,ngram_range=(1,2))
X_train=cv.fit_transform(X_train).toarray()
X_test=cv.transform(X_test).toarray()

In [140]:
cv.vocabulary_

{'one': 1797,
 'register': 2012,
 'enter': 756,
 'draw': 697,
 'gift': 1052,
 'voucher': 2818,
 'reply': 2021,
 'unsubscribe': 2719,
 'text': 2504,
 'stop': 2367,
 'draw gift': 698,
 'gift voucher': 1054,
 'text stop': 2511,
 'lot': 1524,
 'happen': 1211,
 'feel': 840,
 'work': 2929,
 'mo': 1636,
 'hai': 1195,
 'bore': 188,
 'da': 563,
 'last': 1426,
 'weeks': 2882,
 'stuff': 2391,
 'waste': 2849,
 'time': 2573,
 'oh': 1779,
 'well': 2886,
 'later': 1432,
 'yes': 2981,
 'place': 1877,
 'town': 2634,
 'meet': 1591,
 'excite': 801,
 'adult': 23,
 'single': 2214,
 'uk': 2699,
 'txt': 2671,
 'chat': 344,
 'msg': 1681,
 'txt chat': 2675,
 'sorry': 2296,
 'miss': 1627,
 'babe': 127,
 'late': 1431,
 'sleep': 2239,
 'hope': 1289,
 'enjoy': 753,
 'drive': 706,
 'lesson': 1464,
 'boytoy': 199,
 'tease': 2474,
 'kiss': 1406,
 'sorry miss': 2302,
 'sleep hope': 2241,
 'tease kiss': 2475,
 'login': 1502,
 'dat': 575,
 'dad': 566,
 'fetch': 853,
 'home': 1283,
 'double': 693,
 'mins': 1615,
 'price'

In [141]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0

In [142]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)
y_pred=spam_detect_model.predict(X_test)

In [143]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test,y_pred)

0.9901345291479821

In [144]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99       966
        True       0.97      0.95      0.96       149

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [154]:
test_text = "Congratulations! You've been selected to receive a free iPhone 🎉 Click here to claim your prize now!"
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
input = cv.transform(test_text).toarray()
result = spam_detect_model.predict(input)
print(result.astype(int))
print("Spam" if result[0] == 1 else "Ham")

[1]
Spam


### Creating The TF-IDF Model

In [146]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)

In [147]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train=tv.fit_transform(X_train).toarray()
X_test=tv.transform(X_test).toarray()

In [148]:
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0.255, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.333, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.318, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0

In [149]:
tv.vocabulary_

{'urgent': 2274,
 'try': 2191,
 'contact': 404,
 'todays': 2156,
 'draw': 558,
 'show': 1874,
 'prize': 1643,
 'guarantee': 884,
 'call': 230,
 'land': 1109,
 'line': 1175,
 'claim': 333,
 'valid': 2294,
 'hrs': 993,
 'urgent try': 2279,
 'try contact': 2193,
 'contact todays': 410,
 'todays draw': 2157,
 'draw show': 560,
 'show prize': 1875,
 'prize guarantee': 1646,
 'guarantee call': 886,
 'call land': 243,
 'land line': 1110,
 'line claim': 1176,
 'claim valid': 340,
 'valid hrs': 2296,
 'cheer': 317,
 'yeah': 2471,
 'hunny': 1001,
 'wk': 2417,
 'sit': 1899,
 'sound': 1949,
 'havin': 925,
 'gr': 861,
 'fun': 754,
 'keep': 1087,
 'hard': 919,
 'live': 1183,
 'chat': 314,
 'min': 1298,
 'choose': 328,
 'girl': 805,
 'connect': 402,
 'cheap': 315,
 'uk': 2230,
 'service': 1850,
 'bcm': 145,
 'wc': 2366,
 'xx': 2461,
 'wc xx': 2367,
 'yar': 2468,
 'want': 2337,
 'yest': 2484,
 'late': 1123,
 'already': 52,
 'get': 775,
 'ask': 99,
 'go': 818,
 'lor': 1204,
 'still': 1991,
 'act': 14,


In [150]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train, y_train)

In [151]:
#prediction
y_pred=spam_tfidf_model.predict(X_test)

In [107]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9739910313901345


In [108]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       1.00      0.97      0.99       986
        True       0.82      0.98      0.90       129

    accuracy                           0.97      1115
   macro avg       0.91      0.98      0.94      1115
weighted avg       0.98      0.97      0.97      1115



In [156]:
test_text = "Congratulations! You've been selected to receive a free iPhone 🎉 Click here to claim your prize now!"
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
input = tv.transform(test_text).toarray()
result = spam_tfidf_model.predict(input)
print(result.astype(int))
print("Spam" if result[0] == 1 else "Ham")

[1]
Spam
